In [16]:
import pandas as pd

from enum import Enum

In [94]:
class Token(Enum):
    USDT = 1
    BTC = 2
    ETH = 3
    
def token_from_str(line):
    token = line.lower()
    if token == 'usdt':
        return Token.USDT
    elif token == 'eth':
        return Token.ETH
    elif token == 'btc':
        return Token.BTC


class Exchange(Enum):
    BINANCE = 1
    BYBIT = 2
    

def exchange_from_str(line):
    exch = line.lower()
    if exch == 'binance':
        return Exchange.BINANCE
    elif exch == 'bybit':
        return Exchange.BYBIT
    

class BlockchainName(Enum):
    NONE = 1
    BEP20 = 2
    ERC20 = 3
    TRC20 = 4
    

def blockchain_from_str(line):
    bc = line.lower()
    if bc == 'none':
        return BlockchainName.NONE
    elif bc == 'bep20':
        return BlockchainName.BEP20
    elif bc == 'erc20':
        return BlockchainName.ERC20
    elif bc == 'trc20':
        return BlockchainName.TRC20

In [95]:
class Node:
    exchange: Exchange
    token: Token
    money: float

    def __init__(self, exchange, token):
        self.money = 0
        self.exchange = exchange
        self.token = token

    def __str__(self):
        return str(self.exchange) + '-' + str(self.token)

In [96]:
class Edge:
    # selling currency_from to buy currency_to spending trading_fee * currency_from - blockchain_fee money
    node_from: Node
    node_to: Node
    blockchain_name: BlockchainName
    trading_fee: float
    blockchain_fee: float
        
    def __init__(self, node_from, node_to, blockchain_name, trading_fee, blockchain_fee):
        self.node_from = node_from
        self.node_to = node_to
        self.blockchain_name = blockchain_name
        self.trading_fee = trading_fee
        self.blockchain_fee = blockchain_fee

In [97]:
path = '/Users/turmetsmakoev/Desktop/TTS_ddd/turmets/test.csv'

box = pd.read_csv(path)
box

,exchange_from,token_from,exchange_to,token_to,blockchain_name,trading_fee,blockchain_fee
0,Binance,USDT,Bybit,USDT,BEP20,1.000000,0.8
1,Binance,USDT,Bybit,USDT,ERC20,1.000000,3.2
2,Binance,USDT,Bybit,USDT,TRC20,1.000000,0.8
3,Bybit,USDT,Binance,USDT,ERC20,1.000000,4.0
4,Bybit,USDT,Binance,USDT,TRC20,1.000000,0.5
5,Bybit,USDT,Binance,USDT,BEP20,1.000000,0.3
6,Binance,ETH,Binance,BTC,None,0.075688,0.0
7,Binance,BTC,Binance,USDT,None,20663.620000,0.0
8,Binance,ETH,Binance,USDT,None,1563.446540,0.0
9,Bybit,BTC,Bybit,USDT,None,20624.714600,0.0


In [98]:
box

,exchange_from,token_from,exchange_to,token_to,blockchain_name,trading_fee,blockchain_fee
0,Binance,USDT,Bybit,USDT,BEP20,1.000000,0.8
1,Binance,USDT,Bybit,USDT,ERC20,1.000000,3.2
2,Binance,USDT,Bybit,USDT,TRC20,1.000000,0.8
3,Bybit,USDT,Binance,USDT,ERC20,1.000000,4.0
4,Bybit,USDT,Binance,USDT,TRC20,1.000000,0.5
5,Bybit,USDT,Binance,USDT,BEP20,1.000000,0.3
6,Binance,ETH,Binance,BTC,None,0.075688,0.0
7,Binance,BTC,Binance,USDT,None,20663.620000,0.0
8,Binance,ETH,Binance,USDT,None,1563.446540,0.0
9,Bybit,BTC,Bybit,USDT,None,20624.714600,0.0


In [101]:
def get_nodes(data):
    nodes = {} # {Node.str -> Node}
    for i in data:
        row = data[i]
        
        exchange_from = exchange_from_str(row['exchange_from'])
        token_from = token_from_str(row['token_from'])
        exchange_to = exchange_from_str(row['exchange_to'])
        token_to = token_from_str(row['token_to'])
        
        blockchain_name = token_from_str(row['blockchain_name'])
        trading_fee = float(row['trading_fee'])
        blockchain_fee = float(row['blockchain_fee'])
        
        box = Node(exchange_from, token_from)
        nodes[str(box)] = box
        
        box = Node(exchange_to, token_to)
        nodes[str(box)] = box
    return nodes


def get_edges(data, nodes):
    edges = [] # [Edge]
    for i in data:
        row = data[i]
        
        exchange_from = exchange_from_str(row['exchange_from'])
        token_from = token_from_str(row['token_from'])
        node_from = Node(exchange_from, token_from)
        
        exchange_to = exchange_from_str(row['exchange_to'])
        token_to = token_from_str(row['token_to'])
        node_to = Node(exchange_to, token_to)
        
        blockchain_name = token_from_str(row['blockchain_name'])
        trading_fee = float(row['trading_fee'])
        blockchain_fee = float(row['blockchain_fee'])
                
        edge = Edge(nodes[str(node_from)], nodes[str(node_to)], blockchain_name, trading_fee, blockchain_fee)
        edges.append(edge)
    return edges

    

In [102]:
data = box.T.to_dict()
node = get_nodes(data)
edges = get_edges(data, node)
edges

In [13]:
box.T.to_dict()

{0: {'exchange_from': 'Binance',
  'token_from': 'USDT',
  'exchange_to': 'Bybit',
  'token_to': 'USDT',
  'chain_name': 'BEP20',
  'trading_fee': 1.0,
  'blockchain_fee': 0.8},
 1: {'exchange_from': 'Binance',
  'token_from': 'USDT',
  'exchange_to': 'Bybit',
  'token_to': 'USDT',
  'chain_name': 'ERC20',
  'trading_fee': 1.0,
  'blockchain_fee': 3.2},
 2: {'exchange_from': 'Binance',
  'token_from': 'USDT',
  'exchange_to': 'Bybit',
  'token_to': 'USDT',
  'chain_name': 'TRC20',
  'trading_fee': 1.0,
  'blockchain_fee': 0.8},
 3: {'exchange_from': 'Bybit',
  'token_from': 'USDT',
  'exchange_to': 'Binance',
  'token_to': 'USDT',
  'chain_name': 'ERC20',
  'trading_fee': 1.0,
  'blockchain_fee': 4.0},
 4: {'exchange_from': 'Bybit',
  'token_from': 'USDT',
  'exchange_to': 'Binance',
  'token_to': 'USDT',
  'chain_name': 'TRC20',
  'trading_fee': 1.0,
  'blockchain_fee': 0.5},
 5: {'exchange_from': 'Bybit',
  'token_from': 'USDT',
  'exchange_to': 'Binance',
  'token_to': 'USDT',
  'ch